# Greedy Algorithms

A greedy algorithm always makes the choice that looks best at the moment. That is, it makes a locally optimal choice in the hope that this choice will lead to a globally optimal solution.

Procedures:

1. Determine the optimal substructure of the problem.
2. Develop a recursive solution.
3. Show that if we make the greedy choice,then only one subproblem remains.
4. Prove that it is always safe to make the greedy choice.(Steps3 and 4 can occur in either order.)
5. Develop a recursive algorithm that implements the greedy strategy.
6. Convert the recursive algorithm to an iterative algorithm.

Design:

1. Cast the optimization problem as one in which we make a choice and are left with one subproblem to solve.
2. Prove that there is always an optimal solution to the original problem that makes the greedy choice, so that the greedy choice is always safe.
3. Demonstrate optimal substructure by showing that, having made the greedy choice, what remains is a subproblem with the property that if we combine an optimal solution to the subproblem with the greedy choice we have made, we arrive at an optimal solution to the original problem.

## An activity selection problem

\begin{array}{|l|l|l|l|l|l|l|l|l|l|l|l|}
\hline
i & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 \\
\hline
s_i &1&3&0&5&3&5&6&8&8&2&12 \\
\hline
f_i &4&5&6&7&9&9&10&11&12&14&16 \\
\hline
\end{array}

### The optimal substructure

We denote $S_{i j}$ the set of activities that start after $a_i$ finishes and finishes before $a_j$ starts. Suppose we want to find a maximum set of mutually compatible activities in $S_{i j}$ and suppose further that such a maximum set is $A_{i j}$. This set will include some activity $a_k$. By including $a_k$ in an optimal solution, we are left with two subproblems: finding mutually compatible activities in the set $S_{i k}$ (activities that start after activity $a_i$ finishes and that finish before activity $a_k$ starts) and finding mutually compatible activities in the set $S_{kj}$ (activities that start after activity $a_k$ finishes and that finish before activity $a_j$ starts). We have

$$
A_{ij}=A_{ik} \cup \{a_k\} \cup A_{kj}
$$

So the maximum size set $A_{i j}$ of mutually compatible activities in $S_{ij}$ consists of 

$$
\mid A_{i j} \mid=\mid A_{ik} \mid+1+\mid A_{k j} \mid
$$

It can be shown that the optimal solution $A_{i j}$ must also include optimal solutions to the two subproblems for $S_{i k}$ and $S_{k j}$. The argument is as follows: If we could find a set $A_{k j}^{'}$ of mutually compatible activities in $S_{k j}$ where $\mid A_{k j}^{'} \mid > \mid A_{k j} \mid$, then we would have constructed a set of $\mid A_{ik} \mid+1+\mid A_{k j}^{'} \mid>\mid A_{ik} \mid+1+\mid A_{k j} \mid=\mid A_{ij} \mid$ mutually compatible activities, which contradicts the assumption that $A_{i j}$ is optimal.

As a result, we could solve the activity-selection problem by dynamic programming. If we denote the size of an optimal solution for the set $S_{i j}$ by $c[i,j]$, then we would have the recurrence

$$
c[i,j]=c[i,k]+c[k,j]+1
$$

Since we do not know whether $A_{i j}$ will include $a_k$, we have to examine all $a_k$ in $S_{i j}$.

$$
c[i,j]=\Big\{ \begin{array}{ll}
                  0 & S_{i j}=\phi \\
                  \underset{a_k \in S_{i j}}{max}\{ c[i,k]+c[k,j]+1 \} & S_{ij} \neq \phi\\
                \end{array}
$$

We could then develop a recursive algorithm and memoize it, or we could work bottom-up and fill in table entries as we go along. But we would be overlooking another important characteristic of the activity-selection problem that we can use to great advantage.

### Making the greedy choice

The greedy choice is to choose the activity that finishes first to leave as much time as possible for other activities. If we make the greedy choice, we only need to solve one remaining subproblem: finding activities that start after $a_1$ finishes. Let $S_k=\{ a_i \in S: s_i >= f_k \}$ be the set of activities that start after $a_k$ finishes. If we make the greedy choice of activity $a_1$, then $S_1$ remains as the only subproblem to solve. Optimal substructure tells us that if $a_1$ is in the optimal solution, then an optimal solution to the original problem consists of activity $a_1$ and all the activities in an optimal solution to the subproblem $S_1$.

### Correctness of greedy choice

Consider any nonempty subproblem $S_k$, and let $a_m$ be an activity in $S_k$ with the earliest finish time. Then $a_m$ is included in some maximum-size subset of mutually compatible activities of $S_k$.

**Proof**: Let $a_j$ is the activity in $A_k$ with the earliest finish time. If $a_j \neq a_m$, we could always replace $a_j$ with $a_m$ to construct our new $A_k^{'}$ such that $\mid A_k^{'} \mid=\mid A_k \mid$ since by definition $f_m \leq f_j$.

### A recursive greedy algorithm

In order to start, we add the fictitious activity $a_0$ with $f_0=0$, so that subproblem $S_0$ is the entire set of activities $S$.

In [1]:
def rec_act_selector(start,finish,k,n):
    """
    :type start: List[int]
    :type finish: List[int]
    :type k: int
    :type n: int
    :rtype: List[int]
    """
    m=k+1
    # notice that the events are already sorted in finish time
    while m<=n and start[m]<finish[k]:
        m+=1
    if m<=n:
        return [m]+rec_act_selector(start,finish,m,n)
    else:
        return []

In [4]:
start=[1,3,0,5,3,5,6,8,8,2,12]
finish=[4,5,6,7,9,9,10,11,12,14,16]
a0=[0]
rec_act_selector(a0+start,a0+finish,0,len(start))

[1, 4, 8, 11]

### An iterative greedy algorithm

In [10]:
def iter_act_selector(start,finish):
    """
    :type start: List[int]
    :type finish: List[int]
    :type k: int
    :type n: int
    :rtype: List[int]
    """
    m=0
    # artificially set a_0 finish time at 0 so that it is smaller than start[i],i=0,1,...
    last_finish=0
    optimal_solution=[]
    # notice that the events are already sorted in finish time
    while m<len(start):
        if start[m]<last_finish:
            m+=1
        else:
            last_finish=finish[m]
            optimal_solution.append(m)
            m+=1
    return optimal_solution

In [11]:
start=[1,3,0,5,3,5,6,8,8,2,12]
finish=[4,5,6,7,9,9,10,11,12,14,16]
iter_act_selector(start,finish)

[0, 3, 7, 10]

## Minimal Spanning Tree

Given an undirected graph G=(V,E) and a cost $c_e$ for each edge $e$. Find the minimum cost tree $T$ in E that spans all vertices.

Note:

1. T has no cycles.
2. The subgraph (V,T) is connected.

### PRIM'S MINIMUM SPANNING TREE ALGORITHM

* Initialize X=[s] with s in vertices chosen arbitrarily.

* T=empty tree

* while $X!=V$:

      Let e = (u,v) be the cheapest edge of G with u ∈ X, v ∈/ X.
      
      Add e to T.
      
      Add v to X.
      
Runtime analysis: n-1 loops, m edges in each loop, the total is O(m n)

**Implementation with heap**:

* Initialize X=[s] with s in vertices chosen arbitrarily.

* T=empty tree

* key[v]=cheapest edge (u,v) for u in X, where v in V-X (in the heap)

* while $X!=V$:

      Extract min v = e = (u,v) be the cheapest edge of G with u ∈ X, v ∈/ X in the heap

      for each edge (v, w) with w in V-X(in the heap):
       
          delete w from heap
          recompute key[w]=min{key[w],c_{vw}}
          reinsert into the heap
      
      add (u,v) to T.
      
      add v to X.
      
Runtime: n-1 loops. In each loop, extract min O(log n). On the otherhand, there are in total O(m log(n)) time for decrease key. As m>=n-1 since G is connected, the total runtime is O(m log (n)).

### KRUSKAL'S MINIMUM SPANNING TREE ALGORITHM

## HUFFMAN CODES

Goal: Best binary prefix-free encoding for a given set of character frequencies.

Useful fact: Binary codes ↔ Binary trees

Prefix-free ↔ Labelled nodes = the leaves

Input: Probability pi for each character i ∈ Σ.

Notation: If T = tree with leaves ↔ symbols of Σ, then average
encoding length L(T) =  \sum_{i ∈ \Sigma} pi · [depth of i in T]

Output: A binary tree T minimizing the average encoding length L(·).

# Dynamic programming

We typically apply dynamic programming to optimization problems. Such prob- lems can have many possible solutions. Each solution has a value, and we wish to find a solution with the optimal (minimum or maximum) value. We call such a solution an optimal solution to the problem, as opposed to the optimal solution, since there may be several solutions that achieve the optimal value.

Steps:

1. Characterize the structure of an optimal solution.
2. Recursively define the value of an optimal solution.
3. Compute the value of an optimal solution,typically in a bottom-up fashion.
4. Construct an optimal solution from computed information.

If we need only the value of an optimal solution, and not the solution itself, then we can omit step 4. When we do perform step 4, we sometimes maintain additional information during step 3 so that we can easily construct an optimal solution.

## Elements of dynamic programming

### Optimal substructure

### Overlapping subproblems

### Reconstructing an optimal solution

### Memoization

## Rod cutting

Given a rod of length $n$ inches and a table of prices $p_i$ for $i=1,2,...,n$, determine the maximum revenue $r_n$ obtainable by cutting up the rod and selling the pieces. Note that if the price $p_n$ for a rod of length $n$ is large enough, an optimal solution may require no cutting at all.

\begin{array}{|l|l|l|l|l|l|l|l|l|l|l|}
\hline
length\ i & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 \\
\hline
price\ p_i &1&5&8&9&10&17&17&20&24&30 \\
\hline
\end{array}

If an optimal solution cuts the rod into $k$ pieces, for some $1 \leq k \leq n$, then an optimal decomposition

$$
n=i_1+i_2+...+i_k
$$

provides maximum corresponding revenue:

$$
r_n=p_{i_1}+p_{i_2}+...+p_{i_k}
$$

For our sample problem,

* $r_1=1$ from solution $1=1$ (no cuts)
* $r_2=5$ from solution $2=2$ (no cuts)
* $r_3=8$ from solution $3=3$ (no cuts)
* $r_4=10$ from solution $4=2+2$
* $r_5=13$ from solution $5=2+3$
* $r_6=17$ from solution $6=6$ (no cuts)
* $r_7=18$ from solution $7=1+6$
* $r_8=22$ from solution $8=2+6$
* $r_9=25$ from solution $9=3+6$
* $r_{10}=30$ from solution $10=10$ (no cuts)

More generally, we can compute $r_n$ by

$$
r_n=max(p_n, r_1+r_{n-1}, r_2+r_{n-2},...,r_{n-1}+r_1)
$$

This recursion relation implies that in solving the original problem of size $n$, we solve smaller problems of the same type. This is called **optimal substructure**. The above recursion is a bit overcomplicated. In a related but slightly simpler way to arrange a recursive structure:

$$
r_n=\underset{1 \leq i \leq n}{max}(p_i+r_{n-i})
$$

where $r_0=0$

### Recursive top-down implementation

The recursive approach is based on

$$
r_n=\underset{1 \leq i \leq n}{max}(p_i+r_{n-i})
$$

where $r_0=0$

In [81]:
def cut_rod(p,n):
    """
    :type p: List[int]
    :type n: int
    :rtype: int
    """
    if n==0:
        return 0
    else:
        return max([p[i]+cut_rod(p,n-i-1) for i in range(n)])

In [140]:
p=[1,5,8,9,10,17,17,20,24,30,32,40,55,55,70,90,91,92,93,94,95,96]
cut_rod(p,22)

107

The recursive implementation is very inefficient. The reason is because it solves the same subproblems repeatedly. We could analyze the runtime as follows: Let $T(n)$ denote the total number of calls made to cut_rod with length $n$. This expression equals the number of nodes in a subtree whose root is labeled by $n$. Then

$$
T(0)=1
$$

as $r[0]=1$ and

$$
T(n)=1+\sum_{j=0}^{n-1}T(j)
$$

The initial 1 is for the call at the root. Then as a result,

$$
T(n)=2^n
$$

which can be shown by induction.

### Dynamical programming

By storing the results of the subproblems, it is possible to reduce the running time to $O(n^2)$.

**top-down with memoization**: The procedure is still recursive in a natural manner, but modified to save the result of each subproblem (usually in an array or hashtable). The procedure first checks to see whether it has previously solved this subproblem. If so, it returns the saved value, saving further computation at this level; if not, the procedure computes the value in the usual manner. We say that the recursive procedure has been **memoized**; it "remembers" what results it has computed previously.

In [138]:
def memoized_cut_rod(p,n):
    """
    :type p: List[int]
    :type n: int
    :rtype: int
    """
    r=[-1]*n
    return memoized_cut_rod_aux(p,n,r)
    
def memoized_cut_rod_aux(p,n,r):
    if n==0:
        return 0
    if r[n-1]>=0:
        return r[n-1]
    else:
        r[n-1]=max([p[i]+memoized_cut_rod_aux(p,n-i-1,r) for i in range(n)])
        return r[n-1]

In [139]:
p=[1,5,8,9,10,17,17,20,24,30,32,40,55,55,70,90,91,92,93,94,95,96]
memoized_cut_rod(p,22)

107

**bottom-up method** This approach typically depends on some natural notion of the “size” of a subproblem, such that solving any par- ticular subproblem depends only on solving “smaller” subproblems. We sort the subproblems by size and solve them in size order, smallest first. When solving a particular subproblem, we have already solved all of the smaller subproblems its solution depends upon, and we have saved their solutions. We solve each sub- problem only once, and when we first see it, we have already solved all of its prerequisite subproblems.

In [141]:
def bottom_up_cut_rod(p,n):
    """
    :type p: List[int]
    :type n: int
    :rtype: int
    """
    r=[-1]*(n+1)
    r[0]=0
    for i in xrange(1,n+1):
        r[i]=max([p[j-1]+r[i-j] for j in range(1,i+1)])
    return r[1:]

In [142]:
p=[1,5,8,9,10,17,17,20,24,30,32,40,55,55,70,90,91,92,93,94,95,96]
bottom_up_cut_rod(p,22)

[1,
 5,
 8,
 10,
 13,
 17,
 18,
 22,
 25,
 30,
 32,
 40,
 55,
 56,
 70,
 90,
 91,
 95,
 98,
 100,
 103,
 107]

### Exercise

Consider whether the following "greedy" algorithm works: Define the density of a rod of length $i$ to be $p_i/i$. The greedy strategy for a rod of length $n$ cuts off a first piece of length $i$, where $1\leq i\leq n$, having maximum density. It then continues by applying the greedy strategy to the remaining piece of length $n-i$.

## Matrix-chain multiplication

### Exercise:

Give a dynamic-programming algorithm for the activity-selection problem, based on recurrence

$$
c[i,j]=\Big\{ \begin{array}{ll}
                  0 & S_{i j}=\phi \\
                  \underset{a_k \in S_{i j}}{max}\{ c[i,k]+c[k,j]+1 \} & S_{ij} \neq \phi\\
                \end{array}
$$

Have your algorithm compute the sizes $c[i,j]$ as defined above and also produce the maximum-size subset of mutually compatible activities. Assume that the inputs have been sorted based on finish time.

Solution: We first calculate $S_{ij}$, the set of activities that start after $a_i$ finishes and finishes before $a_j$ starts.

$$
S_{ij}=\Big\{\begin{array}{ll}
                  [] & i \geq j \\
                  S_{ij} & i < j \\
                \end{array}
$$

Then we apply the recurrence relation to calculate c[i,j]

In [79]:
class Solution(object):
    def dyn_act_selector(self,start,finish):
        n=len(start)
        S=[[[] for i in range(n)] for j in range(n)]
        A=[[[] for i in range(n)] for j in range(n)]
        for i in range(n-1):
            for j in range(i+1,n):
                S[i][j]=[k for k in range(i+1,j) if start[k]>=finish[i] and finish[k]<=start[j]]
        C=[[0 for i in range(n)] for j in range(n)]
        for i in range(n-1):
            for j in range(i+1,n):
                C[i][j]=self.max_compatible_size(i,j,S)
        for i in range(n-1):
            for j in range(i+1,n):
                A[i][j]=self.max_compatible(i,j,S)
        return C,A
        
    # mutually compatible activities size between i and j
    def max_compatible_size(self,i,j,S):
        if S[i][j]==[]:
            return 0
        else:
            max_size=0
            max_index=None
            for k in S[i][j]:
                size=self.max_compatible_size(i,k,S)+self.max_compatible_size(k,j,S)+1
                if size>max_size:
                    max_size=size
                    max_index=[k]
            return max_size
        
    # mutually compatible activities between i and j
    def max_compatible(self,i,j,S):
        if S[i][j]==[]:
            return []
        else:
            max_size=0
            max_list=[]
            for k in S[i][j]:
                compatible_list=self.max_compatible(i,k,S)+[k]+self.max_compatible(k,j,S)
                if len(compatible_list)>max_size:
                    max_size=len(compatible_list)
                    max_list=compatible_list
            return max_list

In [80]:
o=Solution()
start=[1,3,0,5,3,5,6,8,8,2,12]
finish=[4,5,6,7,9,9,10,11,12,14,16]
C,A=o.dyn_act_selector(start,finish)
C,A

([[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 2],
  [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 2],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[[], [], [], [], [], [], [], [3], [3], [], [3, 7]],
  [[], [], [], [], [], [], [], [3], [3], [], [3, 7]],
  [[], [], [], [], [], [], [], [], [], [], [6]],
  [[], [], [], [], [], [], [], [], [], [], [7]],
  [[], [], [], [], [], [], [], [], [], [], []],
  [[], [], [], [], [], [], [], [], [], [], []],
  [[], [], [], [], [], [], [], [], [], [], []],
  [[], [], [], [], [], [], [], [], [], [], []],
  [[], [], [], [], [], [], [], [], [], [], []],
  [[], [], [], [], [], [], [], [], [], [], []],
  [[], [], [], [], [], [], [], [], [], [], []]])

In [32]:
C=[[0 for i in range(3)] for j in range(3)]
C

[[0, 0, 0], [0, 0, 0], [0, 0, 0]]

In [33]:
max([1,2,3])

3

In [35]:
a=[1,3,5,7]
b=[i for i in a]
b

[1, 3, 5, 7]

# Leetcode

## 45. <font color=red>Jump Game II

Given an array of non-negative integers, you are initially positioned at the first index of the array.

Each element in the array represents your maximum jump length at that position.

Your goal is to reach the last index in the minimum number of jumps.

For example:
Given array A = [2,3,1,1,4]

The minimum number of jumps to reach the last index is 2. (Jump 1 step from index 0 to 1, then 3 steps to the last index.)

Solution: An obvious solution is go backtracking to record the number of minimal jumps from every position to the last index. This algorithm will take O(n^2) time. 

In [314]:
class Solution(object):
    def jump(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        n=len(nums)
        # array keep track of minimal jumps to the end
        r=[-1]*n
        r[n-1]=0
        for i in range(n)[::-1]:
            canjump=False
            min_jump=n-1-i
            for j in range(i+1,n):
                if i+nums[i]>=j and r[j]>=0:
                    canjump=True
                    if r[j]+1<min_jump:
                        min_jump=r[j]+1
                        
            if canjump==True:
                r[i]=min_jump
                
        return r[0]

In [315]:
o=Solution()
o.jump([2,3,1,1,4])

2

The above solution is TLE. Can we do better? Discussion solution: We start from the beginning and make the furthest next jump until the end.

In [323]:
class Solution(object):
    def jump(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        n=len(nums)
        i=0
        j=0
        jump=0
        while j<n-1:
            next_max_pos=min(i+nums[i],n-1)
            while j<=next_max_pos:
                if j+nums[j]>i+nums[i]:
                    i=j
                j+=1
            jump+=1
        return jump

In [319]:
class Solution(object):
    def jump(self, nums):
        i, j, steps = 0, 1, 0
        while j < len(nums):
            endj = min(nums[i]+i+1, len(nums))
            while j < endj:
                if nums[j] + j > nums[i] + i: i = j
                j += 1
            steps += 1
        return steps

In [324]:
o=Solution()
o.jump([2,3,1,1,0])

2

## 55. Jump Game

Given an array of non-negative integers, you are initially positioned at the first index of the array.

Each element in the array represents your maximum jump length at that position.

Determine if you are able to reach the last index.

For example:

A = [2,3,1,1,4], return true.

A = [3,2,1,0,4], return false.

Solution: First observation, the List can be converted into a tree structure, every reachable position from the current position are the children of the current node. Then we search all the way down to the leafs to find whether the last index is in the node of the tree.

In [152]:
class Solution(object):
    def canJump(self, nums):
        """
        :type nums: List[int]
        :rtype: bool
        """
        if nums==[]:
            return True
        # assign the root
        return self.canJumpAux(nums,0)
    
    def canJumpAux(self,nums,pos):
        n=len(nums)
        # assign the root
        root=nums[pos]
        # if the maximum jump reaches the end, return true
        if pos+nums[pos]>=n-1:
            return True
        # search children
        for i in range(pos+1,pos+nums[pos]+1):
            if self.canJumpAux(nums,i):
                return True
            
        return False

In [154]:
o=Solution()
o.canJump([2,3,1,1,4])

True

The above solution is TLE. Let me analyze the time complexity: In the worst case, that is last index is unreachable even when the whole list (except the last) is visited. For position n, let T(n) be the runtime,

$$
T(n)=1+\sum_{i=1}^{nums[n]} T(n+i)
$$

In the worst case, $nums[n]=length-1-n$:

$$
T(n)=1+\sum_{i=1}^{length-1-n} T(n+i) \\
=1+\sum_{i=n+1}^{length-1} T(i)
$$

which turns out to be O(2^length) 

The above solution is purely recursive. We should be able to improve it by storing the results of the subproblems. Let me first consider a top-down approach. There are a lot of repeated subtrees, which means we could label the pos we have already checked by True and False directly. That will take O(length) space.

In [165]:
class Solution(object):
    def canJump(self, nums):
        """
        :type nums: List[int]
        :rtype: bool
        """
        if nums==[]:
            return True
        # assign the root
        n=len(nums)
        r=[None]*len(nums)
        return self.canJumpAux(nums,0,n,r)
    
    def canJumpAux(self,nums,pos,n,r):
        if r[pos]!=None:
            return r[pos]
        # assign the root
        root=nums[pos]
        # if the maximum jump reaches the end, return true
        if pos+nums[pos]>=n-1:
            return True
        # search children
        for i in range(pos+1,pos+nums[pos]+1):
            if self.canJumpAux(nums,i,n,r):
                return True
        
        # label pos as False
        r[pos]=False
        
        return r[pos]

In [174]:
o=Solution()
o.canJump([2,3,1,1,4])

True

Then we consider bottom-up solution. At any position that the jump starts, it can only jump to the right. So we could first label the last index as true, then we go backwards to the left of the list to check whether the nums[i] covers any element that is true. Once we reach the start of the list, we just return r[0].

In [172]:
class Solution(object):
    def canJump(self, nums):
        """
        :type nums: List[int]
        :rtype: bool
        """
        n=len(nums)
        r=[False]*n
        r[n-1]=True
        for i in range(n-2,-1,-1):
            for j in range(i+1,i+nums[i]+1):
                if r[j]==True:
                    r[i]=True
                    
        return r[0]

In [175]:
o=Solution()
o.canJump([2,3,1,1,4])

True

The runtime is O(n^2) and the space is O(n). Can we do better in a Greedy algorithm? We always make the biggest 'backjump' from the last good position

In [254]:
class Solution(object):
    def canJump(self, nums):
        """
        :type nums: List[int]
        :rtype: bool
        """
        n=len(nums)
        last_good=n-1
        for i in range(n)[::-1]:
            if i+nums[i]>=last_good:
                last_good=i
        return last_good==0

In [256]:
o=Solution()
o.canJump([3,2,1,0,4])

False

## 122. Best Time to Buy and Sell Stock II

Say you have an array for which the ith element is the price of a given stock on day i.

Design an algorithm to find the maximum profit. You may complete as many transactions as you like (i.e., buy one and sell one share of the stock multiple times). However, you may not engage in multiple transactions at the same time (i.e., you must sell the stock before you buy again).

Solution: We need to capture all the increasing trend in the array.

In [290]:
class Solution(object):
    def maxProfit(self, prices):
        """
        :type prices: List[int]
        :rtype: int
        """
        buy_time=0
        sell_time=0
        # track if last transaction is closed
        complete=True
        n=len(prices)
        profit=0
        for i in range(0,n):
            if i==0 and i<n-1 and prices[i]<=prices[i+1]:
                buy_time=i
                complete=False
            elif i==n-1 and not complete:
                sell_time=i
                profit+=prices[sell_time]-prices[buy_time]
            elif i>0 and i<n-1:
                if prices[i-1]>=prices[i] and prices[i]<=prices[i+1] and complete:
                    buy_time=i
                    complete=False
                if prices[i-1]<=prices[i] and prices[i]>=prices[i+1] and not complete:
                    sell_time=i
                    profit+=prices[sell_time]-prices[buy_time]
                    complete=True
        return profit

In [296]:
o=Solution()
prices=[1,2,1,1]
o.maxProfit(prices)

1

The above code can be improved as follows:

In [301]:
class Solution(object):
    def maxProfit(self, prices):
        """
        :type prices: List[int]
        :rtype: int
        """
        n=len(prices)
        profit=0
        # edge case
        if n<=1:
            return profit
        buy_time=0
        sell_time=0
        # track if last transaction is closed
        if prices[0]<=prices[1]:
            complete=False
        else:
            complete=True
        for i in range(1,n-1):
            if prices[i-1]>=prices[i] and prices[i]<=prices[i+1] and complete:
                buy_time=i
                complete=False
            if prices[i-1]<=prices[i] and prices[i]>=prices[i+1] and not complete:
                sell_time=i
                profit+=prices[sell_time]-prices[buy_time]
                complete=True
        # complete any unclosed deal
        if not complete:
            sell_time=n-1
            profit+=prices[sell_time]-prices[buy_time]
        return profit

In [302]:
o=Solution()
prices=[1]
o.maxProfit(prices)

0

There turns out to be a single pass solution as we only need to add up profit[i+1]>profit[i].

## 134. Gas Station

There are N gas stations along a circular route, where the amount of gas at station i is gas[i].

You have a car with an unlimited gas tank and it costs cost[i] of gas to travel from station i to its next station (i+1). You begin the journey with an empty tank at one of the gas stations.

Return the starting gas station's index if you can travel around the circuit once, otherwise return -1.

Note:
The solution is guaranteed to be unique.

Solution: Suppose the car starts at 0, if it finishes the circle, then

gas[0]-cost[0]>0

gas[0]-cost[0]+gas[1]-cost[1]>0

...

gas[0]-cost[0]+gas[1]-cost[1]+...gas[n]-cost[n]>0

We could examine the net gain of gas for each link. The job can be done in a single pass: we start at 0, if 

gas[0]-cost[0]+gas[1]-cost[1]+...gas[i]-cost[i]<0 for some i

then start=i+1 and if the remaining gain from i+1 to n can compensate for the loss, then we return the starting position.

In [306]:
class Solution(object):
    def canCompleteCircuit(self, gas, cost):
        """
        :type gas: List[int]
        :type cost: List[int]
        :rtype: int
        """
        n=len(gas)
        gain=[gas[i]-cost[i] for i in range(n)]
        start=0
        tank=0
        compensate=0
        for i in range(n):
            if tank+gain[i]<0:
                start=i+1
                compensate+=tank+gain[i]
                tank=0
            else:
                tank+=gain[i]
        return start if tank+compensate>=0 else -1

In [309]:
o=Solution()
o.canCompleteCircuit([5,3],[4,1])

0

## 455. Assign Cookies

Assume you are an awesome parent and want to give your children some cookies. But, you should give each child at most one cookie. Each child i has a greed factor $g_i$, which is the minimum size of a cookie that the child will be content with; and each cookie j has a size $s_j$. If $s_j \geq g_i$, we can assign the cookie j to the child i, and the child i will be content. Your goal is to maximize the number of your content children and output the maximum number.

Note:

You may assume the greed factor is always positive.

You cannot assign more than one cookie to one child.

Example 1:

Input: [1,2,3], [1,1]

Output: 1

Explanation: You have 3 children and 2 cookies. The greed factors of 3 children are 1, 2, 3. 
And even though you have 2 cookies, since their size is both 1, you could only make the child whose greed factor is 1 content.
You need to output 1.

Example 2:

Input: [1,2], [1,2,3]

Output: 2

Explanation: You have 2 children and 3 cookies. The greed factors of 2 children are 1, 2. 
You have 3 cookies and their sizes are big enough to gratify all of the children, 
You need to output 2.


Solution: Discussion solution: sort both lists; assign cookies from the least greedy child. Why does this work? If for the first child, $g_1>s_1$, then we have to move on to the cookie that could satisfy the first child; The same applies to the second child. Why is this the best case? Because if the cookies are assigned to anyone with high greediness, it can be assigned to the ones with low greediness without decreasing the total number of content children. Similarly, if one child is assigned with a large size satisfying his greediness, he could also be assigned with a smaller size cookie without decreasing the total number of content children.

In [ ]:
class Solution(object):
    def findContentChildren(self, g, s):
        """
        :type g: List[int]
        :type s: List[int]
        :rtype: int
        """
        # sort lists
        g.sort()
        s.sort()
        # assign cookies to children
        i=0
        j=0
        content_number=0
        while i<len(g) and j<len(s):
            if s[j]>=g[i]:
                content_number+=1
                i+=1
                j+=1
            else: # not satsifying the child, move on to the next larger size cookie
                j+=1
                
        return content_number

## 452. Minimum Number of Arrows to Burst Balloons

There are a number of spherical balloons spread in two-dimensional space. For each balloon, provided input is the start and end coordinates of the horizontal diameter. Since it's horizontal, y-coordinates don't matter and hence the x-coordinates of start and end of the diameter suffice. Start is always smaller than end. There will be at most 104 balloons.

An arrow can be shot up exactly vertically from different points along the x-axis. A balloon with xstart and xend bursts by an arrow shot at x if xstart ≤ x ≤ xend. There is no limit to the number of arrows that can be shot. An arrow once shot keeps travelling up infinitely. The problem is to find the minimum number of arrows that must be shot to burst all balloons.

Example:

Input:
[[10,16], [2,8], [1,6], [7,12]]

Output:
2

Explanation:
One way is to shoot one arrow for example at x = 6 (bursting the balloons [2,8] and [1,6]) and another arrow at x = 11 (bursting the other two balloons).


Solution: A greedy algorithm is to make every shot as effective as possible. We can first sort the list according to start time. Then we find the region for the first shoot. Remove the balloons bursts and then find the region for the second shoot...until all burst.

In [194]:
class Solution(object):
    def findMinArrowShots(self, points):
        """
        :type points: List[List[int]]
        :rtype: int
        """
        if points==[]:
            return 0
        # sort the points
        points.sort()
        mark=points[0]
        count=1
        for i in range(1,len(points)):
            if points[i][0]>mark[1]:
                count+=1
                mark=points[i]
            else:
                mark[1]=min(points[i][1],mark[1])
        return count

In [195]:
o=Solution()
o.findMinArrowShots([[1,10],[2,3],[3,5],[6,7]])

2

## 406. Queue Reconstruction by Height

Suppose you have a random list of people standing in a queue. Each person is described by a pair of integers (h, k), where h is the height of the person and k is the number of people in front of this person who have a height greater than or equal to h. Write an algorithm to reconstruct the queue.

Note:
The number of people is less than 1,100.

Example

Input:
[[7,0], [4,4], [7,1], [5,0], [6,1], [5,2]]

Output:
[[5,0], [7,0], [5,2], [6,1], [4,4], [7,1]]


Solution: Discussion solution: 

1. Sort first by height in descending order, then by index in ascending order. 
2. Insert elements in the second tallest group from left to right into the first tallest group according to their index. (As there will be no taller elements afterwards.)
3. Repeat the procedure for 3rd, 4th ... tallest group.

To realize the above algorithm we create a hashtable with height as key and lists of indices as values.

In [226]:
class Solution(object):
    def reconstructQueue(self, people):
        """
        :type people: List[List[int]]
        :rtype: List[List[int]]
        """
        height_dict={}
        for e in people:
            if height_dict.get(e[0]):
                height_dict[e[0]].append(e[1])
            else:
                height_dict[e[0]]=[e[1]]
        heights=sorted(height_dict.keys(),reverse=True)
        result=[]
        for i in range(len(heights)):
            if i==0:
                height_dict[heights[i]].sort()
                result+=[[heights[i],key] for key in height_dict[heights[i]]]
            else:
                height_dict[heights[i]].sort()
                for pos in height_dict[heights[i]]:
                    result.insert(pos,[heights[i],pos])
        return result

In [227]:
o=Solution()
people=[[7,0], [4,4], [7,1], [5,0], [6,1], [5,2]]
o.reconstructQueue(people)

[[5, 0], [7, 0], [5, 2], [6, 1], [4, 4], [7, 1]]

## 402. Remove K Digits

Given a non-negative integer num represented as a string, remove k digits from the number so that the new number is the smallest possible.

Note:
The length of num is less than 10002 and will be ≥ k.
The given num does not contain any leading zero.

Example 1:

Input: num = "1432219", k = 3
Output: "1219"
Explanation: Remove the three digits 4, 3, and 2 to form the new number 1219 which is the smallest.

Example 2:

Input: num = "10200", k = 1
Output: "200"
Explanation: Remove the leading 1 and the number is 200. Note that the output must not contain leading zeroes.

Example 3:

Input: num = "10", k = 2
Output: "0"
Explanation: Remove all the digits from the number and it is left with nothing which is 0.


Solution: Given k=1, what should be removed first? We should remove the largest digit, but if the highest digit is followed with bunch of zeros, we should count them as one. For k>2, we remove the largest digit by the same method in the remaining number.

In [ ]:
class Solution(object):
    def removeKdigits(self, num, k):
        """
        :type num: str
        :type k: int
        :rtype: str
        """
        

<font color=red>
The above idea is wrong take the example: 13425 for k=1, we have to remove 4 instead of 5. The right consideration is as follows: for s[i],i=0,1,2,... in num

* if $s[i]<s[i+1]$: pass as remove s[i] will be larger than remove s[i+1]; if $s[i]>=s[i+1]$: remove s[i]. Notice that for k>=1, we need to move to the previous position such that all the k digits can be removed in one pass.


* but if there is more to be removed, we just remove the remaining digits from the end. As the remaining digits in the number are strictly in ascending order.

* Lastly, we need to take care of the leading zeros.

In [252]:
class Solution(object):
    def removeKdigits(self, num, k):
        """
        :type num: str
        :type k: int
        :rtype: str
        """
        n=len(num)
        i=0
        t=0
        num=list(num)
        result=[]
        while i<n:
            result.append(num[i])
            while t-1>=0 and result[t-1]>result[t] and k>0:
                result.pop(t-1)
                k-=1
                t-=1
            i+=1
            t+=1
        # check if there is more to be removed from the end
        for i in range(k):
            result.pop()
        #check starting zeros
        n=len(result)
        start=0
        while start<=n-1 and result[start]=='0':
            start+=1
            
        return '0' if start>n-1 else ''.join(result[start:])

In [253]:
o=Solution()
o.removeKdigits('101',2)

'0'

Notice that the corner case where all digits have been removed should be considered and the output is '0'.

## 435. Non-overlapping Intervals

Given a collection of intervals, find the minimum number of intervals you need to remove to make the rest of the intervals non-overlapping.

Note:
* You may assume the interval's end point is always bigger than its start point.
* Intervals like [1,2] and [2,3] have borders "touching" but they don't overlap each other.

Example 1:
Input: [ [1,2], [2,3], [3,4], [1,3] ]

Output: 1

Explanation: [1,3] can be removed and the rest of intervals are non-overlapping.

Example 2:
Input: [ [1,2], [1,2], [1,2] ]

Output: 2

Explanation: You need to remove two [1,2] to make the rest of intervals non-overlapping.

Example 3:
Input: [ [1,2], [2,3] ]

Output: 0

Explanation: You don't need to remove any of the intervals since they're already non-overlapping.


Solution: Intuitively, we will first remove the one that has the most number of overlaps with others, then the second most overlap interval...Specifically,

1. sort the start of intervals: O(n log n)
2. check if two intervals are overlapping: Brutal force: O(n^2), make a hashtable to record the overlapping intervals for each interval.
3. remove the one with the largest list and then move on to the second largest...

The above algorithm is not a good greedy algorithm. Instead, after sorting the intervals according to the starting point, we could remove the one with larger end in the overlapping pair of intervals as it is more likely to overlap the intervals afterwards.

In [8]:
# Definition for an interval.
class Interval(object):
    def __init__(self, s=0, e=0):
        self.start = s
        self.end = e

class Solution(object):
    def eraseOverlapIntervals(self, intervals):
        """
        :type intervals: List[Interval]
        :rtype: int
        """
        intervals.sort(key=lambda x: x.start)
        n=len(intervals)
        if n==0: return 0
        prev_end=intervals[0].end
        count=0
        for i in xrange(1,n):
            if prev_end>intervals[i].start: # overlapping
                count+=1
                prev_end=min(prev_end,intervals[i].end)
            else: # non-overlapping
                prev_end=intervals[i].end
        return count

In [7]:
o=Solution()
interval1=Interval()
interval1.start=1
interval1.end=2
interval2=Interval()
interval2.start=2
interval2.end=3
intervals=[interval1,interval2]
o.eraseOverlapIntervals(intervals)

0